<a href="https://colab.research.google.com/github/MaxGSEO/Cloud-page-semantic-publishing/blob/main/GSC_Analysis_%2B_Content_Decay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Search Console Analysis (Use Case 2)


In [ ]:
#install this to save plotly pictures
%%capture
!pip install advertools plotly -U kaleido

You may need to restart your runtime after this installation.

From the menu above, Runtime > Restart runtime and ignore this cell!



In [ ]:
# data
import pandas as pd
import numpy as np

# viz
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import kaleido.scopes.plotly as kaleido

# utils
import datetime

Please recall to change "sample.csv" with the name of the CSV file you have uploaded.

How to upload a file? It's explained in the ebook.

In [ ]:
#upload your GSC file from the API
main = pd.read_csv("data_sample.csv")
#create copy of dataframe just in case
df = main.copy()

In [ ]:
#number of rows of the dataframe
len(df)

# Data cleaning

In [ ]:
#remove all URLs containing the following strings. You can add/remove them based on the website you are analyzing.
filter_page = ['category', '#', 'blog', 'tag', 'author', 'wp-content', 'upload', 'page']

df = df[~df['page'].str.contains("|".join(filter_page))]

In [ ]:
#removing foreign characters
def is_ascii(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

#this line of code applies the function to the query column of the dataframe.
df = df[df['query'].map(lambda x: x.isascii())]

In [ ]:
len(df.page.unique())

In [ ]:
df.drop_duplicates(['query', 'page', 'date'], inplace=True)

# Creating new metrics

In [ ]:
#keywords a page is ranking for
df['n_kws_page_ranks_for'] = df['page'].map(df.groupby('page')['page'].count())
rows = len(df)
df['percent_kws_page_ranks_for'] = (df['n_kws_page_ranks_for'] / rows * 100).round(2)

#feature engineering
df.fillna({"impressions": 0, "clicks": 0, "position": 0}, inplace=True)
df = df.astype({"impressions": int, "clicks": int, "position": int})
#create new colummns
df['total_clicks_page'] = df['clicks'].groupby(df['page']).transform('sum')
df['total_page_imps_percent'] = df['impressions'].groupby(df['page']).transform('sum')
#percentages
df['total_page_clicks_percent'] = (df['total_clicks_page'] / sum(df['clicks']) * 100).round(2)
df['page_percent_imps'] = (df['total_page_imps_percent'] / sum(df['impressions']) * 100).round(2)
df = df.round(2)

# Get leading queries by clicks or impressions (i.e. the top query by page)

We can add even more columns to enhance our analysis. I love inserting 2 columns to find what are the best queries for every page.

This allows us to understand if the page is "matching" its intent.

E.g. An article not performing well will show queries that are barely related, in most cases.

In [ ]:
#add leading queries
df.drop_duplicates(inplace=True)

# Group the data by the URL and sort the queries by the number of clicks
df_clicks = df.groupby('page')['query', 'clicks'].apply(lambda x: x.sort_values('clicks', ascending=False))

# Select the first row of each group (the query with the highest number of clicks)
df_clicks = df_clicks.groupby('page').first()


# Group the data by the URL and sort the queries by the number of impressions
df_impressions = df.groupby('page')['query', 'impressions'].apply(lambda x: x.sort_values('impressions', ascending=False))

# Select the first row of each group (the query with the highest number of impressions)
df_impressions = df_impressions.groupby('page').first()


In [ ]:
df['leading_query_by_clicks'] = df['page'].map(df_clicks['query'])
df['leading_query_by_impressions'] = df['page'].map(df_impressions['query'])

# Some DataViz

We apply Sturges' Rule to find the ideal number of bins for the histogram.

In [ ]:
# find number of bins with Sturges' Rule
n_bins = 1 + int(np.log(len(df)))

The snippet of code below is classifiyng queries according to how many clicks they get.

To do so, we have to group by query first and then sum clicks.

In [ ]:
#create bands based on clicks. Having more values in low groups means you are weaker to avg. position drops.
def label_bands (row):
   if row['clicks'] == 0:
      return '0-click'
   if row['clicks'] == 1:
      return '1-click'
   if 2 <= row['clicks'] <= 20:
      return '2-to-20'
   if 21 <= row['clicks'] <= 100:
      return '21-100'
   return '100+'

grpd = df.groupby(["query"]).agg({"clicks": "sum"}).sort_values(by="clicks", ascending=False)
grpd['clicks_category'] = grpd.apply(lambda x: label_bands(x), axis=1)
df = df.merge(right=grpd.reset_index()[['query', 'clicks_category']], on="query", how="inner")

Same operation for Impressions.

In [ ]:
#create bands based on impressions.
def label_bands_impressions (row):
   if row['impressions'] == 1:
      return '1-imp'
   if 2 <= row['impressions'] <= 20:
      return '2-to-20'
   if 21 <= row['impressions'] <= 100:
      return '21-100'
   return '100+'

grpd = df.groupby(["query"]).agg({"impressions": "sum"}).sort_values(by="impressions", ascending=False)
grpd['impressions_category'] = grpd.apply(lambda x: label_bands_impressions(x), axis=1)
df = df.merge(right=grpd.reset_index()[['query', 'impressions_category']], on="query", how="inner")

We are ready to plot a histogram displaying a full breakdown of the queries on a website.

In [ ]:
clicks_band_hist = px.histogram(
    data_frame=df.drop_duplicates(['query', 'clicks_category', 'impressions_category']),
    x='clicks_category',
    y='query',
    nbins=n_bins,
    title="Query Count by Clicks - Finding opportunities + Assessing Status",
    labels={'clicks_category': 'Clicks Band', 'count':'Frequency'},
    template='plotly_dark',
    width=1024,
    height=600, histfunc="count", color="impressions_category"
)
clicks_band_hist.update_layout( # customize font and legend orientation & position
    font_family="Inter",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center",
    ),
    xaxis={'categoryorder':'total descending'}
)
clicks_band_hist.show()

We can also measure query count over time with a simple line plot.

In [ ]:
df['month'] = df.date.dt.month

def plot_clicks_by_month(df):
  # create a new column with the month for each date

  # group the data by query band and month, and sum the number of clicks
    df = df.groupby(['clicks_category', 'month'])['query'].nunique().reset_index()

  # use Plotly to create a line chart with the sum of clicks on the y-axis
  # and the month on the x-axis, and a separate line for each query band
    fig = px.line(df, x='month', y='query', color='clicks_category',
               template='plotly_dark', width=1024, height=600, labels={'month': 'Month', 'query':'Unique query count'})
    fig.update_xaxes(dtick="M1", tickformat="%Y-%m")
    fig.update_yaxes(autorange=True)
    fig.show()
    fig.write_image("query_over_time.png")

plot_clicks_by_month(df)


# Queries with 0 clicks

In [ ]:
df_zero_query = df.loc[df['clicks_category'] == '0-click'][['query','position']]
df_zero_query.drop_duplicates(['query'], inplace=True)
df_zero_query.to_csv("zero_clicks_queries.csv")

# Find bi/tri-grams

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))
# matrix of ngrams
ngrams = c_vec.fit_transform(df[df['position'] < 10].sort_values("clicks", ascending=True)['query'].head(500))
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
           ).rename(columns={0: 'frequency', 1:'bigram/trigram'})

# Calculate cosine similarity between ngrams
similarity_matrix = cosine_similarity(ngrams.T)

# Set a similarity threshold
similarity_threshold = 0.8

# Create a list of similar ngrams to remove
ngrams_to_remove = set()
for i in range(len(similarity_matrix)):
    for j in range(i+1, len(similarity_matrix)):
        if similarity_matrix[i,j] > similarity_threshold:
            ngrams_to_remove.add(i)
            ngrams_to_remove.add(j)

# Remove similar ngrams from the dataframe
df_ngram_filtered = df_ngram.loc[~np.isin(range(len(df_ngram)), list(ngrams_to_remove))]
df_ngram_filtered.head(100).style.background_gradient()


# Percentage of 0-clicks pages

In [ ]:
total = len(df.groupby('page').clicks.sum())
zero_clicks = len(np.where(df.groupby('page').clicks.sum() == 0)[0])
zero_clicks_perc = round( (zero_clicks / total) * 100, 2)
zero_clicks_perc

In [ ]:
#create bands based on clicks. Having more values in low groups means you are weaker to avg. position drops.
def label_bands (row):
   if row['clicks'] == 0:
      return '0-click'
   if row['clicks'] == 1:
      return '1-click'
   if 2 <= row['clicks'] <= 20:
      return '2-to-20'
   if 21 <= row['clicks'] <= 100:
      return '21-100'
   return '100+'

grpd = df.groupby(["page"]).agg({"clicks": "sum"}).sort_values(by="clicks", ascending=False)
grpd['clicks_pages'] = grpd.apply(lambda x: label_bands(x), axis=1)
df = df.merge(right=grpd.reset_index()[['page', 'clicks_pages']], on="page", how="inner")

In [ ]:
zero_pages = pd.Series(df[df['clicks_pages'] == "0-click"]['page'].unique())
zero_pages.to_csv("zero_pages.csv")

In [ ]:
top_10_pages_clicks_sum = df.groupby('page').clicks.sum().sort_values(ascending=False).head(10).sum()
all_clicks_sum =  df.groupby('page').clicks.sum().sum()
top_10_pages_clicks_perc = round((top_10_pages_clicks_sum / all_clicks_sum) * 100, 2)
top_10_pages_clicks_perc

# What's the contribution of the top 10 pages (by Clicks)?

In [ ]:
top_10_pages_clicks_sum = df.groupby('page').clicks.sum().sort_values(ascending=False).head(10).sum()
all_clicks_sum =  df.groupby('page').clicks.sum().sum()
top_10_pages_clicks_perc = round((top_10_pages_clicks_sum / all_clicks_sum) * 100, 2)
top_10_pages_clicks_perc

In [ ]:
#you can also get the contribution of the top 1%. This refers to the 99th percentile!
top_1_perc = np.percentile(df.groupby('page').clicks.sum(), 99) # Calculate the 99th percentile
top_1_pages_clicks_sum = df.groupby('page').clicks.sum()[df.groupby('page').clicks.sum() >= top_1_perc].sum() # Calculate the clicks generated by pages in the top 1%
all_clicks_sum =  df.groupby('page').clicks.sum().sum()
top_1_pages_clicks_perc = round((top_1_pages_clicks_sum / all_clicks_sum) * 100, 2)
top_1_pages_clicks_perc

# Heatmap

A heatmap is just a way to visualize a pivot table.

You can even skip it but I don't recommend doing it because it looks cooler.

In [ ]:
#please recall that we created a month variable from date in Use Case 2.
df.date = pd.to_datetime(df.date)

In [ ]:
#keeping all queries which position is less than or equal to 10. N.B. this is a quick and inaccurate way to filter.
#hm_data_qc_pos_in_time = df.query('position <= 10').pivot_table(index='position', columns='month', values = "query", aggfunc='count', fill_value=0)
#this line of code takes unique queries into account. You can use the above line if you want to consider all of them instead.
hm_data_qc_pos_in_time = df.query('position <= 10').pivot_table(index='position', columns='month', values = "query", aggfunc=lambda x: len(set(x)), fill_value=0)
#renaming numbers into month names
hm_data_qc_pos_in_time.rename(columns={
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}, inplace=True)

In [ ]:
heatmap = px.imshow(hm_data_qc_pos_in_time, text_auto=True)
heatmap.update_layout(
     title='Change of positions over time',
     yaxis = dict(
        tickmode = 'array',
        tickvals = [i for i in np.arange(11)],
    ),
     template='plotly_dark',
     xaxis_title='',
     yaxis_title='Rankings',
     width=1024,
     height=600,
     legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
        ))
heatmap.show()

# Classifying Pages Based On Performance

Ideally, you shouldn't give advice based on all the pages of a website. Why?

Because it's not actionable.

This is why I want to show you a basic but effective way of classifying pages.

P.S. The idea below was inspired by [Daniel Foley Carter](https://seo-audits.io/).


In [ ]:
#create bands based on clicks. Having more values in low groups means you are weaker to avg. position drops.
def label_bands (row):
   if row['clicks'] > 1000:
      return 'Top'
   elif 101 <= row['clicks'] <= 1000 :
      return 'Good'
   elif 21 <= row['clicks'] <= 100:
      return 'Fair'
   elif 1 <= row['clicks'] <= 20:
      return 'Weak'
   elif row ['clicks'] == 0 and row['impressions'] > 100:
      return 'Opportunity'
   elif row ['clicks'] == 0 and row['impressions'] <= 100:
      return 'Dead'

grpd = df.groupby(["page"]).agg({"clicks": "sum", "impressions":"sum"}).sort_values(by="clicks", ascending=False)
grpd['groups_pages'] = grpd.apply(lambda x: label_bands(x), axis=1)
df = df.merge(right=grpd.reset_index()[['page', 'groups_pages']], on="page", how="inner")

In the next updates, I will show you a much better way. The example above is using arbitrary numbers to show you how it's done.

In practice, you want something more reliable like percentiles to spot the top 1%, 20%, 50% etc.

Since the top 20% would also include the top 1%, you have to be careful when doing this. Don't worry, I will explain how in the next versions.

In [ ]:
# you never know...
unique_df = df.drop_duplicates(subset="page")

# select "page" and "groups_pages" columns
result = unique_df[["page", "groups_pages"]]

In [ ]:
# let's color the groups! You can change the values as you wish but be careful: everything has a meaning.
color_map = {'Weak': '#FF6347', 'Dead': '#FF0000',
'Top': '#32CD32', 'Good': '#7CFC00',
'Fair': '#ADFF2F', 'Opportunity': '#9ACD32'}

# Create a histogram
fig = px.histogram(
    x=result['groups_pages'],
    color=result['groups_pages'],
    color_discrete_map=color_map,
    #nbins=len(counts),
    title='Page Classes Based on Performance',
    labels={'x': 'Page Groups', 'y': 'Frequency'},
    template='plotly_dark',
    color_discrete_sequence=px.colors.qualitative.Dark2,
)

# Customize the layout
fig.update_layout(
    font_family='Inter',
    legend=dict(
        title=None, orientation='h', y=1, yanchor='bottom', x=0.5, xanchor='center',
    ),
    xaxis=dict(categoryorder='total descending')
)

# Show the histogram
fig.show()

If you want some text to copy and paste in your report (maybe in a bullet list?), look no further.

In [ ]:
# Define the class values to iterate over
class_values = ['Opportunity', 'Weak', 'Good', 'Fair', 'Top', 'Dead']
total_pages = len(result)

# Iterate over the class values and print the results for each one
for class_value in class_values:
    # Calculate the number of pages in the current class value
    class_pages = len(result[result['groups_pages'] == class_value])

    # Calculate the percentage of pages in the current class value out of the total
    percent_class = (class_pages / total_pages) * 100

    # Print the result using an f-string
    print(f"{class_pages} out of {total_pages} pages ({percent_class:.2f}%) are {class_value}.")


# Content Decay (Use Case 3)

Content Decay is quite simple to score, you just need to load the same data as before.

This part is not separate, it actually continues what's done before!


In [ ]:
#create new time variables
df['month'] = df.date.dt.month
df['day'] = df.date.dt.day
#grouping and aggregating clicks
g = df.groupby(['page','month']).agg({'clicks': 'sum'})
#now you have a list of non-duplicate pages
unique_pages = list(g.reset_index().page.unique())

In [ ]:
#initialize empty list
container = []
#loop over the list unique_pages with an index
for i, page in enumerate(unique_pages):
    res = {} #initialize empty dictionary
    subset = g.loc[page]
    x = list(subset.index)
    y = subset.clicks.values

    # fit model
    try:
        slope, intercept = np.poly1d(np.polyfit(x, y, 1)) #get slope and intercept of the line
        slope = round(slope, 2) #round slope values to 2 digits
    except: #if there is an error, set the slope to None
        slope = None
        intercept = None

    #assign values to the empty dictionary we initialited before
    res['page'] = page
    res['slope'] = slope
    container.append(res)

#convert the dictionary into a dataframe
slope_df = pd.DataFrame(container).sort_values(by='slope', ascending=True)
slope_df.head(25)

Disclaimer: if you get the following error: "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: RankWarning:

Polyfit may be poorly conditioned"

is fine!

Python is just telling you that the line is not based on enough data points, which is not a problem for our scenario.

We are just interested in pages displaying extremely negative values.

A better alternative is using quantiles and finding the top 10% of pages.

In [ ]:
#arbitrary value, can use what you want
len(slope_df[slope_df['slope'] < -10])

In [ ]:
#save your dataframe to CSV
slope_df.reset_index().head(30).to_csv("decaying_pages.csv")